Refer to the Landsat Collection 2 Level 3 DSWe data format control book ([direct link](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-2042_LandsatC2_L3_DSWE_DFCB-v2.pdf), [USGS landing page](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-collection-2-level-3-dynamic-surface-water-extent)) for information on the various DSWE data products

In [1]:
from pystac_client import Client
import pystac
import geopandas as gpd
import json
import netrc
import requests
from pathlib import Path
from bs4 import BeautifulSoup
from typing import List

In [2]:
usgs_stac_url = 'https://landsatlook.usgs.gov/stac-server'
catalog = Client.open(usgs_stac_url)

In [3]:
# We have a reference shapefile describing a polygon over Lake Mead in Nevada
df = gpd.read_file('../data/shapefiles/lake_mead/aoi.shp')
shape_geojson = json.loads(df.to_json())['features'][0]['geometry']

In [4]:
shape_geojson

{'type': 'Polygon',
 'coordinates': [[[-114.87222049842939, 36.16273394123082],
   [-114.50459507920661, 36.22948555290927],
   [-114.50643320630273, 36.53876281932189],
   [-114.02668203421706, 36.54023965625615],
   [-114.02852016131315, 35.975526918105764],
   [-114.86670611714104, 36.00973286261627],
   [-114.87222049842939, 36.16273394123082]]]}

Information about options that can be specified to a STAC API ItemSearch call can be [found here](https://pystac-client.readthedocs.io/en/latest/api.html#item-search)

In [5]:
opts = {
    'intersects' : shape_geojson,
    'collections':'landsat-c2l3-dswe',
    'datetime' : "2015-12-01/2020-12-31",
    'max_items' : 10
}

search = catalog.search(**opts)
items = search.get_all_items()
len(items)

10

In [6]:
print(items[0].assets.keys())

dict_keys(['index', 'diag', 'intr', 'intsm', 'inwam', 'mask', 'shade', 'xml', 'json'])


Using code from [here](https://gist.github.com/scottyhq/ae2a53ecc955c5957c27c35d3bd9bce2) to finally download the data

In [7]:
def download_asset(item:List[pystac.item.Item], asset_type:List[str]=['diag'], download_path:str='../data/downloads'):

    if type(item) is not list : item = [item]
    if type(asset_type) is not list : asset_type = [asset_type]

    download_path = Path(download_path)
    download_path.mkdir(exist_ok = True)
    
    creds = netrc.netrc()
    user,account,password = creds.authenticators('ers.cr.usgs.gov')

    url = 'https://ers.cr.usgs.gov/login'
    with requests.Session() as s:
        
        r = s.get(url)
        soup = BeautifulSoup(r.content, 'html.parser') 
        sval = soup.find('input', attrs={'name':'csrf'})['value']

        data = {"username": user, 
            "password": password,
            "csrf": sval}

        print('Authenticating with netrc...')
        bf = s.post(url, data = data)
        print(bf.status_code)

    
    for _item in item:
        for _asset_type in asset_type:
            asset_url = _item.assets[_asset_type.lower()].href
            filename = asset_url.split('/')[-1]

            print(f'Downloading {filename}...')
            respb = s.get(asset_url,
                        allow_redirects=True,
                        headers = {'content-type': 'image/tiff'})

            with open(Path(download_path) / filename, 'wb') as src:
                src.write(respb.content)

            print(f"File downloaded to {Path(download_path) / filename}")
    

In [8]:
# We can download the default ('diagnostic') asset type
download_asset(items[5])

Authenticating with netrc...
200
File downloaded to ../data/downloads/LE07_CU_006011_20201213_20210504_02_DIAG.TIF


In [9]:
# We can specify an alternative asset type to download
download_asset(items[7], asset_type='mask')

Authenticating with netrc...
200
File downloaded to ../data/downloads/LC08_CU_006011_20201212_20210504_02_MASK.TIF


In [10]:
# We can specify a list of items and a list of asset types to download
download_asset(items[2:4], asset_type=['intr', 'intsm', 'inwam'])

Authenticating with netrc...
200
File downloaded to ../data/downloads/LC08_CU_006011_20201228_20210504_02_INTR.TIF
File downloaded to ../data/downloads/LC08_CU_006011_20201228_20210504_02_INTSM.TIF
File downloaded to ../data/downloads/LC08_CU_006011_20201228_20210504_02_INWAM.TIF
File downloaded to ../data/downloads/LE07_CU_005011_20201220_20210504_02_INTR.TIF
File downloaded to ../data/downloads/LE07_CU_005011_20201220_20210504_02_INTSM.TIF
File downloaded to ../data/downloads/LE07_CU_005011_20201220_20210504_02_INWAM.TIF
